### numerical만 포함하기


In [3]:
import pandas as pd
import numpy as np

df_origin_trn = pd.read_csv('/home/sangjun/work/data/HN__4_8__19__train.csv') #임시 -csv직접 올려서 사용함
df_origin_tst = pd.read_csv('/home/sangjun/work/data/HN__2020__19__selected.csv') # 2020년도만 분리한 테스트셋 

In [4]:
num_features = ["age",
"ainc",
"HE_PLS_15",
"HE_PLS_30",
"HE_mPLS",
"HE_sbp",
"HE_dbp",
"HE_ht",
"HE_wt",
"HE_wc",
"HE_glu",
"HE_HbA1c",
"HE_insulin",
"HE_chol",
"HE_HDL_st2",
"HE_TG",
"HE_LDL_drct",
"HE_ast",
"HE_alt",
"HE_HB",
"HE_HCT",
"HE_BUN",
"HE_crea",
"HE_WBC",
"HE_RBC",
"HE_Bplt",
"HE_Uacid",
"HE_Uph",
"HE_Usg",
"HE_Ucrea"]


In [5]:
y_related =["BP_PHQ_1",
"BP_PHQ_2",
"BP_PHQ_3",
"BP_PHQ_4",
"BP_PHQ_5",
"BP_PHQ_6",
"BP_PHQ_7",
"BP_PHQ_8",
"BP_PHQ_9",
"mh_PHQ_S",
"BP6_10",
"BP6_31",
"DF2_pr",
"DF2_pt",
"BP1"]

In [6]:
print(df_origin_trn.shape)
print(df_origin_tst.shape)

(87986, 125)
(6133, 125)


In [7]:
#TODO 
"""DF2_pt 변수의 고려
depressed == 0 & mh_PHQ_S != NaN & DF2_pr ==1: 
치료 여부를 파악해서 고려해주어야 함"""

'DF2_pt 변수의 고려\ndepressed == 0 & mh_PHQ_S != NaN & DF2_pr ==1: \n치료 여부를 파악해서 고려해주어야 함'

In [8]:
def manual_preprocess(df:pd.DataFrame, num_features, y_related):
    # 1) 9, 99, 999 인 녀석들(모름/무응답) null 처리
    df = df.replace([9, 99, 999, 9999], np.nan, inplace=False)

    # 2) mh_PHQ_S 는 null값인데, 각 문항 응답인 BP_PHQ_1 ~ 9의 값이 있는 경우, 더해서 mh_PHQ_S에 넣어주기
    cond = df["BP_PHQ_1"].notna() & \
            df["BP_PHQ_2"].notna() & \
            df["BP_PHQ_3"].notna() & \
            df["BP_PHQ_4"].notna() & \
            df["BP_PHQ_5"].notna() & \
            df["BP_PHQ_6"].notna() & \
            df["BP_PHQ_7"].notna() & \
            df["BP_PHQ_8"].notna() & \
            df["BP_PHQ_9"].notna()
    
    a = df[df["mh_PHQ_S"].isnull() & cond][["BP_PHQ_1", "BP_PHQ_2", "BP_PHQ_3", "BP_PHQ_4", "BP_PHQ_5", "BP_PHQ_6", "BP_PHQ_7", "BP_PHQ_8", "BP_PHQ_9"]]
    b = a[a <= 3].sum(axis=1)
    b[b >= 1] #125

    df['mh_PHQ_S'] = b[b >=1]

    # 3) 결측값 처리 1 - row 기준
    threshold = 0.3 # threshold for null proportion
    null_percent = df.isnull().mean(axis = 1) # 각 행에 대한 결측값 비율 계산
    df = df[null_percent < threshold]
    
    # 4) 변수 생성 `depressed` 
    cond1 = df['mh_PHQ_S'] >= 10 # mh_PHQ_S - total score 
    cond2 = df['BP_PHQ_9'].isin([1, 2, 3]) # person who chose 1, 2, 3 in 9th question 
    cond3 = df['BP6_10'] == 1 
    cond4 = df['BP6_31'] == 1
    cond5 = (df['DF2_pr'] == 1) & (df['mh_PHQ_S'].isna())

    df[cond1 | cond2 | cond3 | cond4 | cond5].drop_duplicates()
 
    df.loc[cond1 | cond2 | cond3 | cond4 | cond5, 'depressed'] = 1 # 조건 중 하나 이상을 만족하는 행에 'depressed' 변수에 1 값 
    df.loc[~(cond1 | cond2 | cond3 | cond4 | cond5), 'depressed'] = 0 # 조건 중 하나라도 만족하지 않는 행에 0 값

    # 5) 학습에 방해가 될 수 있는 그룹 제거 (우울증에 관한 정보가 없는 경우)
    df.drop(df[df['DF2_pr'].isna() & df['mh_PHQ_S'].isna()].index, inplace=True)
    df = df[(df['DF2_pr'] != 8 |  ~df['mh_PHQ_S'].isna())]

    # 6) 결측값 처리 2 - columns 기준
    threshold = 0.1 # 10% 기준 # TODO 30%로 바꿔보기 
    null_percent = df.isnull().mean() # 각 열에 대한 결측값 비율 계산
    selected_cols = null_percent[null_percent < threshold].index
    df = df[selected_cols]

    # 7) y_related 변수를 피쳐에서 제외하기
    remained_y = set(df.columns).intersection(set(y_related))
    y = df['depressed'] # y 생성
    X = df.drop(columns=list(remained_y), inplace=False) # X 생성

    # 8) 위의 결측값 처리에서 빠진 column을 각 피쳐 그룹에서 제외하기
    num_features = set(df.columns).intersection(set(num_features))
#     onehot_features = set(df.columns).intersection(set(onehot_features))
#     label_features = set(df.columns).intersection(set(label_features))

#     # 9) label features중 비해당(8) 값을 각각 알맞은 값으로 대체하기
#     label_ans_8 = ["DI3_2", "BD1_11", "BD2_11", "BA2_12", "BA2_13", "BP1", "BS3_1","BE3_31", "BE5_1"] # BD2_11 - removed

#     # 'DI3_2' 변수의 값이 8인 경우, 해당 값을 0으로 바꾸기
#     X['DI3_2'] = X['DI3_2'].apply(lambda x: 3 if x == 8 else x)
#     X['BD1_11'] = X['BD1_11'].apply(lambda x: 1 if x == 8 else x)
#     # X['BD2_11'] = X['BD2_11'].apply(lambda x: 0 if x == 8 else x)
#     X['BA2_12'] = X['BA2_12'].apply(lambda x: 5 if x == 8 else x)
#     X['BA2_13'] = X['BA2_13'].apply(lambda x: 5 if x == 8 else x)
#     X['BA2_13'] = X['BA2_13'].apply(lambda x: 5 if x == 8 else x)
#     # X['BP1'] = X['BP1'].apply(lambda x: 5 if x == 8 else x)
#     X['BS3_1'] = X['BS3_1'].apply(lambda x: 0 if x == 8 else x)
#     # X['BE3_31'] = X['BE3_31'].apply(lambda x: 5 if x == 88 else x)
#     # X['BE5_1'] = X['BE5_1'].apply(lambda x: 5 if x == 88 else x)

    print("X.shape:", X.shape)
    print("y.shape:", y.shape)

    return X, y, list(num_features), list(y_related)

In [9]:
trn_X, trn_y, num_features, y_related = manual_preprocess(df_origin_trn, num_features, y_related)
tst_X, tst_y, num_features, y_related = manual_preprocess(df_origin_tst, num_features, y_related)

X.shape: (74517, 89)
y.shape: (74517,)
X.shape: (5226, 99)
y.shape: (5226,)


In [10]:
# 10) train column과 test column 일치시키기
tst_X = tst_X[list(trn_X.columns)]
tst_X.shape

(5226, 89)

In [11]:
### 임시 -transform
def transform(X, y, num_features, y_related):
    ### 결측값 처리 3 - fillna: KNN
    from sklearn.impute import KNNImputer
    from sklearn.impute import SimpleImputer

    # 1) num_features 결측값 처리 - KNN
    knn_imp = KNNImputer(n_neighbors=5)
    X[num_features] = knn_imp.fit_transform(X[num_features])

    # # 2) onehot_features, label_features 결측값 처리 - 최빈값(most frequent values)
    # freq_imp = SimpleImputer(strategy = "most_frequent")
    # X[onehot_features] = freq_imp.fit_transform(X[onehot_features])
    # X[label_features] = freq_imp.fit_transform(X[label_features])

    # transform array(X) to dataframe
    df_new = pd.DataFrame(X, columns=X.columns) 
    df_new.isna().sum().sort_values(ascending = False)

    ### 인코딩
    # 1. numeric # 1. numeric - scaling 
    from sklearn.preprocessing import MinMaxScaler 
    scaler = MinMaxScaler(feature_range=(0, 1))
    df_new[num_features] = scaler.fit_transform(df_new[num_features])
    # # 2. onehot -> onehot 인코딩으로 진행
    # df_new = pd.get_dummies(df_new, columns=onehot_features) # sklearn OneHotEncoder 사용도 가능함

    ### 샘플링
    # under sampling.  OVER 만 하기엔 너무 많이 생성해야해서 under 선수행
    from imblearn.under_sampling import RandomUnderSampler

    X = df_new
    rus = RandomUnderSampler(sampling_strategy=0.5, random_state=42)
    X_under, y_under = rus.fit_resample(X, y)
    X_under.shape, y_under.shape

    # over sampling
    from imblearn.over_sampling import SMOTE

    smote = SMOTE(random_state=42)
    X_over, y_over = smote.fit_resample(X_under, y_under)

    X_over.shape, y_over.shape

    return X_over, y_over


In [13]:
X_over, y_over = transform(trn_X[num_features], trn_y, num_features, y_related)
#override
X_over = trn_X[num_features]
y_over = trn_y

/tmp/ipykernel_355474/1080958715.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[num_features] = knn_imp.fit_transform(X[num_features])


In [14]:
X_test_over, y_test_over = transform(tst_X[num_features], tst_y, num_features, y_related)
#override
X_test_over = tst_X[num_features]
y_test_over = tst_y

/tmp/ipykernel_355474/1080958715.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[num_features] = knn_imp.fit_transform(X[num_features])


In [15]:
X_over.shape, y_over.shape,

((74517, 22), (74517,))

In [16]:
X_test_over.shape, y_test_over.shape

((5226, 22), (5226,))

In [17]:
X_over.to_csv('/home/sangjun/work/', index=False)
y_over.to_csv('/home/sangjun/work/', index=False)

# X_test_over.to_csv('../data/HN_X_231206_numerical_wo_transform_test.csv', index=False)
# y_test_over.to_csv('../data/HN_y_231206_numerical__test.csv', index=False)

OSError: Cannot save file into a non-existent directory: '../data'